In [ ]:
!pip install dwave-ocean-sdk matplotlib pandas --quiet


In [ ]:
from google.colab import files
uploaded = files.upload()
import pandas as pd

qubo_df = pd.read_csv("lonere_exam_delivery_qubo.csv")
Q = {(row["Var1"], row["Var2"]): row["Value"] for _, row in qubo_df.iterrows()}


In [ ]:
from dwave.system import LeapHybridSampler
import dimod

bqm = dimod.BinaryQuadraticModel.from_qubo(Q)
sampler = LeapHybridSampler()
result = sampler.sample(bqm, time_limit=5)

sample = result.first.sample
energy = result.first.energy

import pandas as pd
pd.DataFrame(list(sample.items()), columns=["Variable", "Value"]).to_csv("qubo_solution.csv", index=False)
print("Best solution energy:", energy)


In [ ]:
import matplotlib.pyplot as plt

locations = ["Lonere", "Pune", "Aurangabad", "Nagpur"]
coords = {
    "Lonere": (73.6, 18.1),
    "Pune": (73.85, 18.52),
    "Aurangabad": (75.32, 19.88),
    "Nagpur": (79.09, 21.15)
}

# Decode QUBO sample
position_to_city = {}
for var, val in sample.items():
    if val == 1:
        _, i, j = var.split('_')
        position_to_city[int(j)] = locations[int(i)]

route = [position_to_city[i] for i in sorted(position_to_city.keys())]

# Plot route
plt.figure(figsize=(10, 7))
for city in route:
    x, y = coords[city]
    plt.plot(x, y, 'o', markersize=10)
    plt.text(x + 0.05, y + 0.05, city, fontsize=10)

for i in range(len(route) - 1):
    x1, y1 = coords[route[i]]
    x2, y2 = coords[route[i+1]]
    plt.plot([x1, x2], [y1, y2], 'r--')
# Return to depot
x1, y1 = coords[route[-1]]
x2, y2 = coords[route[0]]
plt.plot([x1, x2], [y1, y2], 'r--', linestyle='dotted')

plt.title("Optimized Route from QUBO")
plt.grid(True)
plt.show()


In [ ]:
from itertools import permutations

distance = {
    ("Lonere", "Pune"): 110,
    ("Lonere", "Aurangabad"): 390,
    ("Lonere", "Nagpur"): 587,
    ("Pune", "Aurangabad"): 240,
    ("Pune", "Nagpur"): 945,
    ("Aurangabad", "Nagpur"): 420
}
# Make symmetric
for (a, b), d in list(distance.items()):
    distance[(b, a)] = d

def route_cost(route):
    return sum(distance.get((route[i], route[i+1]), 0) for i in range(len(route)-1)) + distance.get((route[-1], route[0]), 0)

best_route, min_cost = None, float("inf")
for perm in permutations(locations[1:]):
    r = ["Lonere"] + list(perm)
    cost = route_cost(r)
    if cost < min_cost:
        min_cost = cost
        best_route = r

print("Brute-force optimal route:", best_route)
print("Travel time (minutes):", min_cost)
